In [1]:
import torch
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

/home/kobayashi/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler

# 1. Load the autoencoder model which will be used to decode the latents into image space. 
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae",torch_dtype=torch.float16)

# # 2. Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# # 3. The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.15.mlp.fc2.bias', 'vision_model.encoder.layers.18.self_attn.q_proj.bias', 'vision_model.encoder.layers.9.self_attn.q_proj.bias', 'vision_model.encoder.layers.4.mlp.fc2.bias', 'vision_model.encoder.layers.7.self_attn.q_proj.weight', 'vision_model.encoder.layers.22.self_attn.out_proj.bias', 'vision_model.encoder.layers.18.mlp.fc2.weight', 'vision_model.encoder.layers.3.mlp.fc2.weight', 'vision_model.encoder.layers.4.mlp.fc2.weight', 'vision_model.encoder.layers.17.layer_norm1.weight', 'vision_model.encoder.layers.15.self_attn.k_proj.weight', 'vision_model.encoder.layers.21.self_attn.v_proj.bias', 'vision_model.encoder.layers.5.mlp.fc1.weight', 'vision_model.encoder.layers.8.self_attn.q_proj.weight', 'vision_model.encoder.layers.22.layer_norm2.weight', 'vision_model.encoder.layers.14.mlp.fc2.bias', 'vision_model.encoder.layers.19.self_attn.v_pr

In [13]:
from diffusers import LMSDiscreteScheduler

scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

In [14]:
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)

In [15]:
prompt1 = ["a photograph of an astronaut riding a horse","a photograph of an astronaut riding a bike"]
# prompt2 = [
# prompt1 = ["a cat","a dog"]

height = 384                        # default height of Stable Diffusion
width = 384                         # default width of Stable Diffusion

num_inference_steps = 50            # Number of denoising steps

guidance_scale = 7.5                # Scale for classifier-free guidance

generator = torch.manual_seed(32)   # Seed generator to create the inital latent noise

batch_size = 1


#########################################


text_input1 = tokenizer(prompt1, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
# text_input2 = tokenizer(prompt2, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")


with torch.no_grad():
  text_embeddings = text_encoder(text_input1.input_ids.to(torch_device))[0]

# with torch.no_grad():
#   text_embeddings2 = text_encoder(text_input2.input_ids.to(torch_device))[0]
#########################################

#絵の比率調整
#一点交差
#text_embeddings[0][:8] = text_embeddings2[0][:8]#text_embeddings2が前半
#それぞれの配列で交差
# for i in range(77):#384が真ん中768が最大
#     text_embeddings[0][i][:600] = text_embeddings2[0][i][:600]#text_embeddings2が前半
#########################################

max_length = text_input1.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)
with torch.no_grad():
  uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

###############################################

embeddings = torch.cat([uncond_embeddings, text_embeddings])

#############################################

latents_torch = torch.randn(
  (batch_size, unet.in_channels, height // 8, width // 8),
  generator=generator,
)
latents = latents_torch.to(torch_device)

################################################

latents.shape

#############################################

scheduler.set_timesteps(num_inference_steps)

##########################################

latents_1 = latents * scheduler.init_noise_sigma

#############################################

from tqdm.auto import tqdm
from torch import autocast
from PIL import Image

for t in tqdm(scheduler.timesteps):
  # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
  latent_model_input_a = torch.cat([latents_1] * 2)

  latent_model_input = scheduler.scale_model_input(latent_model_input_a, t)

  # predict the noise residual
  with torch.no_grad():
    noise_pred = unet(latent_model_input, t, encoder_hidden_states=embeddings).sample

  # perform guidance
  noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
  noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

  # compute the previous noisy sample x_t -> x_t-1
  latents_1 = scheduler.step(noise_pred, t, latents_1).prev_sample

###############################################

# scale and decode the image latents with vae
latents_3 = 1 / 0.18215 * latents_1

with torch.no_grad():
  image = vae.decode(latents_3).sample


##########################################

image_1 = (image / 2 + 0.5).clamp(0, 1)
image_2 = image_1.detach().cpu().permute(0, 2, 3, 1).numpy()
images = (image_2 * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
result_image = pil_images[0]
result_image.save("sample.png")

  0%|          | 0/50 [00:00<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 76>                                                                               │
│                                                                                                  │
│    81                                                                                            │
│    82   # predict the noise residual                                                             │
│    83   with torch.no_grad():                                                                    │
│ ❱  84 │   noise_pred = unet(latent_model_input, t, encoder_hidden_states=embeddings).sample      │
│    85                                                                                            │
│    86   # perform guidance                                                                       │
│    87   noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)                                 │
│                                                                                                  │
│ /home/kobayashi/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/modules/module.py:11 │
│ 90 in _call_impl                                                                                 │
│                                                                                                  │
│   1187 │   │   # this function, and just call forward.                                           │
│   1188 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1189 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1190 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1191 │   │   # Do not call functions when jit is used                                          │
│   1192 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1193 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /home/kobayashi/anaconda3/envs/pytorch/lib/python3.9/site-packages/diffusers/models/unet_2d_cond │
│ ition.py:381 in forward                                                                          │
│                                                                                                  │
│   378 │   │   down_block_res_samples = (sample,)                                                 │
│   379 │   │   for downsample_block in self.down_blocks:                                          │
│   380 │   │   │   if hasattr(downsample_block, "has_cross_attention") and downsample_block.has   │
│ ❱ 381 │   │   │   │   sample, res_samples = downsample_block(                                    │
│   382 │   │   │   │   │   hidden_states=sample,                                                  │
│   383 │   │   │   │   │   temb=emb,                                                              │
│   384 │   │   │   │   │   encoder_hidden_states=encoder_hidden_states,                           │
│                                                                                                  │
│ /home/kobayashi/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/modules/module.py:11 │
│ 90 in _call_impl                                                                                 │
│                                                                                                  │
│   1187 │   │   # this function, and just call forward.                                           │
│   1188 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1189 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1190 │   │   │   return forward_call(*input, **kwargs)   

SyntaxError: invalid syntax (18319358.py, line 1)

In [2]:
prompt = ["a photograph of an astronaut riding a horse"]

height = 512                        # default height of Stable Diffusion
width = 512                         # default width of Stable Diffusion

num_inference_steps = 100            # Number of denoising steps

guidance_scale = 7.5                # Scale for classifier-free guidance

generator = torch.manual_seed(32)   # Seed generator to create the inital latent noise

batch_size = 1

#########################################


text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

#text_inputの値を変更
# text_input.input_ids[0][8]=4500

with torch.no_grad():
  text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]


#############################################

max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)
with torch.no_grad():
  uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

###############################################

embeddings = torch.cat([uncond_embeddings, text_embeddings])

#############################################

latents_torch = torch.randn(
  (batch_size, unet.in_channels, height // 8, width // 8),
  generator=generator,
)
latents = latents_torch.to(torch_device)

################################################

latents.shape

#############################################

scheduler.set_timesteps(num_inference_steps)

##########################################

latents_1 = latents * scheduler.init_noise_sigma

#############################################

from tqdm.auto import tqdm
from torch import autocast
from PIL import Image

for t in tqdm(scheduler.timesteps):
  # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
  latent_model_input_a = torch.cat([latents_1] * 2)

  latent_model_input = scheduler.scale_model_input(latent_model_input_a, t)

  # predict the noise residual
  with torch.no_grad():
    noise_pred = unet(latent_model_input, t, encoder_hidden_states=embeddings).sample

  # perform guidance
  noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
  noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

  # compute the previous noisy sample x_t -> x_t-1
  latents_1 = scheduler.step(noise_pred, t, latents_1).prev_sample

###############################################

# scale and decode the image latents with vae
latents_3 = 1 / 0.18215 * latents_1

with torch.no_grad():
  image = vae.decode(latents_3).sample


##########################################

image_1 = (image / 2 + 0.5).clamp(0, 1)
image_2 = image_1.detach().cpu().permute(0, 2, 3, 1).numpy()
images = (image_2 * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
result_image = pil_images[0]
result_image.save("sample.png")



NameError: name 'torch' is not defined

torch.Size([1, 4, 64, 64])

In [44]:
#ファイルに書き込み
import numpy
writing = embeddings.to('cpu').detach().numpy()
for i in range(len(writing)):
    numpy.savetxt("matrix/"+prompt[0]+"_embeddings"+str(i)+".txt",writing[i])
numpy.savetxt("matrix/"+prompt[0]+"_input"+".txt",text_input.input_ids)

-0.2880389988422394


In [43]:
text_input

{'input_ids': tensor([[49406,   320,  8853,   539,   550, 18376,  6765,   320,  3701, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])}